In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Head_and_Neck_Cancer"
cohort = "GSE218109"

# Input paths
in_trait_dir = "../../input/GEO/Head_and_Neck_Cancer"
in_cohort_dir = "../../input/GEO/Head_and_Neck_Cancer/GSE218109"

# Output paths
out_data_file = "../../output/preprocess/Head_and_Neck_Cancer/GSE218109.csv"
out_gene_data_file = "../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE218109.csv"
out_clinical_data_file = "../../output/preprocess/Head_and_Neck_Cancer/clinical_data/GSE218109.csv"
json_path = "../../output/preprocess/Head_and_Neck_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Esophageal Squamous Cell Carcinoma tumors from Indian patients: nuclear-stabilized p53 (NS+) versus unstable p53 (NS-) protein"
!Series_summary	"Transcriptional profiling of Esophageal Squamous Cell Carcinoma (ESCC) tumors comparing samples harbouring nuclear-stabilized p53 (NS+) versus unstable p53 (NS-) protein, determined through immunohistochemistry (IHC) staining of the tumor sections. The goal was to identify the genes that were differentially regulated between NS+ and NS- ESCC samples."
!Series_overall_design	"Two-condition experiment, NS+ versus NS- esophageal tumors. NS+ tumors: 17, NS- tumors: 19."
Sample Characteristics Dictionary:
{0: ['Sex: M', 'Sex: F'], 1: ['age: 22', 'age: 45', 'age: 52', 'age: 50', 'age: 34', 'age: 55', 'age: 48', 'age: 64', 'age: 70', 'age: 68', 'age: 23', 'age: 62', 'age: 59', 'age: 58', 'age: 41', 'age: 47', 'age: 66', 'age: 38', 'age: 79', 'age: 61', 'age: 39', 'age: 32', 'age: 46', 'age: 69', 'age: 54'], 2: [

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Dict, Any, Optional, Callable

# 1. Gene Expression Data Availability Analysis
# Based on the background information, this dataset appears to contain gene expression data
# as it mentions "Transcriptional profiling of Esophageal Squamous Cell Carcinoma (ESCC) tumors"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Trait: The trait here appears to be related to p53 status (NS+ vs NS-) from index 5
trait_row = 5

# Age: Available at index 1
age_row = 1

# Gender: Available at index 0
gender_row = 0

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert p53 status to binary: 1 for NS+, 0 for NS-"""
    if not isinstance(value, str):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'nuclear-stabilized p53 (NS+)' in value:
        return 1
    elif 'unstable p53 (NS-)' in value:
        return 0
    return None

def convert_age(value):
    """Convert age to continuous numeric value"""
    if not isinstance(value, str):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male"""
    if not isinstance(value, str):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Initial filtering and saving metadata
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if is_trait_available:
    # Assuming clinical_data was loaded in a previous step
    try:
        # Assuming clinical_data exists from previous step
        clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Ensure output directory exists
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        # If there's an error, we still want to proceed with the rest of the pipeline


Error in clinical feature extraction: [Errno 2] No such file or directory: '../../input/GEO/Head_and_Neck_Cancer/GSE218109/clinical_data.csv'


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Head_and_Neck_Cancer/GSE218109/GSE218109_series_matrix.txt.gz
Gene data shape: (31214, 36)
First 20 gene/probe identifiers:
Index(['12', '14', '15', '16', '17', '18', '19', '20', '22', '23', '24', '25',
       '26', '27', '30', '33', '35', '36', '37', '38'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers from the previous output

# The identifiers shown are numeric values like '12', '14', '15', etc.
# These are not standard human gene symbols, which would typically be alphanumeric 
# like 'BRCA1', 'TP53', 'EGFR', etc.
# 
# These appear to be probe IDs or some other numeric identifiers that would need 
# to be mapped to actual gene symbols for meaningful biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'COL', 'ROW', 'NAME', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'GENE', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'TIGR_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE', 'SPOT_ID.1', 'ORDER']
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['266', '266', '266', '266', '266'], 'ROW': [170.0, 168.0, 166.0, 164.0, 162.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [n

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which column in gene annotation maps to gene expression data identifiers
# From the previous outputs, we can see:
# - Gene expression data has numeric IDs like '12', '14', '15', etc. 
# - Gene annotation has an 'ID' column, which contains numeric values
# - We need gene symbols which are in the 'GENE_SYMBOL' column

# Extract gene identifier and gene symbol columns
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'
print(f"Using '{prob_col}' column as probe ID and '{gene_col}' column as gene symbol")

# 2. Get gene mapping dataframe 
gene_mapping = get_gene_mapping(gene_annotation, prob_col=prob_col, gene_col=gene_col)
print(f"Gene mapping shape: {gene_mapping.shape}")
print("First 5 rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe data to gene expression data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)
print(f"After mapping, gene data shape: {gene_data.shape}")
print("First 10 mapped gene symbols:")
print(gene_data.index[:10])

# Check the quality of mapping - how many genes have been successfully mapped
print(f"Number of unique genes after mapping: {len(gene_data.index)}")

# Normalize gene symbols to standardize them
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization, gene data shape: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Using 'ID' column as probe ID and 'GENE_SYMBOL' column as gene symbol
Gene mapping shape: (32696, 2)
First 5 rows of gene mapping:
    ID          Gene
11  12      APOBEC3B
13  14        ATP11B
14  15  LOC100132006
15  16        DNAJA1
17  18         EHMT2
After mapping, gene data shape: (15298, 36)
First 10 mapped gene symbols:
Index(['A1BG', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'AAAS', 'AACS',
       'AADAC', 'AADACL1'],
      dtype='object', name='Gene')
Number of unique genes after mapping: 15298
After normalization, gene data shape: (14998, 36)
First 10 normalized gene symbols:
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'AAAS', 'AACS', 'AADAC',
       'AADAT', 'AAK1'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE218109.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        # Check if trait is biased
        trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
        if trait_type == "binary":
            is_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            is_biased = judge_continuous_variable_biased(linked_data, trait)
            
        # Remove biased demographic features
        if "Age" in linked_data.columns:
            age_biased = judge_continuous_variable_biased(linked_data, 'Age')
            if age_biased:
                linked_data = linked_data.drop(columns='Age')
                
        if "Gender" in linked_data.columns:
            gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
            if gender_biased:
                linked_data = linked_data.drop(columns='Gender')
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to Randall's plaque tissue, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data from Randall's plaque tissue associated with kidney stones."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for kidney stones analysis."
    )
    
    print("Dataset is not usable for kidney stones analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (14998, 36)
Gene data shape after normalization: (14998, 36)


Normalized gene expression data saved to ../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE218109.csv
Original clinical data preview:
         !Sample_geo_accession                                  GSM6734720  \
0  !Sample_characteristics_ch1                                      Sex: M   
1  !Sample_characteristics_ch1                                     age: 22   
2  !Sample_characteristics_ch1  tissue: Esophageal Squamous Cell Carcinoma   
3  !Sample_characteristics_ch1                                Stage: pT3N2   
4  !Sample_characteristics_ch1                                    grade: I   

                                   GSM6734721  \
0                                      Sex: M   
1                                     age: 45   
2  tissue: Esophageal Squamous Cell Carcinoma   
3                                Stage: pT3N0   
4                                    grade: I   

                                   GSM6734722  \
0                                      Sex: F

Data shape after handling missing values: (36, 15001)
For the feature 'Head_and_Neck_Cancer', the least common label is '1.0' with 17 occurrences. This represents 47.22% of the dataset.
Quartiles for 'Age':
  25%: 44.0
  50% (Median): 53.0
  75%: 62.0
Min: 22.0
Max: 79.0
For the feature 'Gender', the least common label is '0.0' with 14 occurrences. This represents 38.89% of the dataset.


Linked data saved to ../../output/preprocess/Head_and_Neck_Cancer/GSE218109.csv
